In [1]:
import torch.nn as nn 
from torch import optim
from torch.utils.data import DataLoader
import torchvision 
from PIL import Image 
from catalyst import dl

In [2]:
class ConvAutoencoder(nn.Module): 
    def __init__(self): 
        super(ConvAutoencoder, self).__init__() 
        self.encoder = nn.Sequential( 
            nn.Conv2d(1, 16, 4), 
            nn.ReLU(), 
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(16, 4, 4), 
            nn.ReLU(), 
            nn.AvgPool2d(9), 
        ) 
        self.decoder = nn.Sequential( 
            nn.ConvTranspose2d(4, 16, 5, stride=2), 
            nn.ReLU(), 
            nn.ConvTranspose2d(16, 4, 5, stride=2), 
            nn.ReLU(), 
            nn.ConvTranspose2d(4, 1, 4, stride=2), 
            nn.Sigmoid(), 
        ) 

    def forward(self, x): 
        bottleneck_feature = self.encoder(x) 
        reconstructed_x = self.decoder( 
          bottleneck_feature 
        ) 
        return reconstructed_x 

In [3]:
class FashionMNISTImageTarget( 
    torchvision.datasets.FashionMNIST 
): 
    def __getitem__(self, index): 
        img = self.data[index]         
        img = Image.fromarray( 
            img.numpy(), mode="L" 
        )         
        if self.transform is not None: 
            img = self.transform(img) 
        return img, img 

In [4]:
def transform_image(image): 
    return torchvision.transforms.ToTensor()(image) 

In [5]:
train_fashion_mnist_data = FashionMNISTImageTarget( 
    'fashion_mnist/', download=True, train=True, transform=transform_image, 
) 
valid_fashion_mnist_data = FashionMNISTImageTarget( 
    'fashion_mnist/', download=True, train=False, transform=transform_image, 
) 
loaders = { 
    "train": DataLoader( 
        train_fashion_mnist_data, batch_size=32, shuffle=True 
    ), 
    "valid": DataLoader( 
        valid_fashion_mnist_data, batch_size=32 
    ), 
} 

In [6]:
criterion = nn.MSELoss() 

In [7]:
runner = dl.SupervisedRunner( 
    input_key="features", output_key="scores", target_key="targets", loss_key="loss" 
) 

In [8]:
def train_and_evaluate_mlp( 
    trial_number, net, epochs, load_on_stage_start=False, best_or_last='last', verbose=False 
): 
    model = net 
    optimizer = optim.Adam( 
    model.parameters(), lr=0.02 
    )     
    checkpoint_logdir = "logs/trial_{}_autoencoder".format(trial_number) 
    runner.train( 
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        loaders=loaders, 
        num_epochs=epochs, 
        callbacks=[ 
            dl.CheckpointCallback( 
                logdir=checkpoint_logdir, 
                loader_key="valid", 
                metric_key="loss", 
                load_on_stage_end='best', 
            ) 
        ], 
        logdir="./logs", 
        valid_loader="valid", 
        valid_metric="loss", 
        minimize_valid_metric=True, 
        verbose=verbose, 
    ) 
    with open(os.path.join(checkpoint_logdir, '_metrics.json'), 'r' ) as f: 
        metrics = json.load(f) 
    if best_or_last == 'last': 
        valid_loss = metrics['last']['_score_'] 
    else: 
        valid_loss = metrics['best']['valid']['loss'] 
    return valid_loss 

In [ ]:
cnn_autoencoder = ConvAutoencoder() 
best_valid_loss = train_and_evaluate_mlp( 
    0, cnn_autoencoder, 20, load_on_stage_start=False, best_or_last='last', verbose=True 
) 

/home/dcek/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


1/20 * Epoch (train):   0%|          | 0/1875 [00:00<?, ?it/s]

train (1/20) loss: 0.06007776159048081 | loss/mean: 0.06007776159048081 | loss/std: 0.009008542275007882 | lr: 0.02 | momentum: 0.9


1/20 * Epoch (valid):   0%|          | 0/313 [00:00<?, ?it/s]

valid (1/20) loss: 0.05547902113199235 | loss/mean: 0.05547902113199235 | loss/std: 0.006020873103538906 | lr: 0.02 | momentum: 0.9
* Epoch (1/20) 


...

4/20 * Epoch (train):   0%|          | 0/1875 [00:00<?, ?it/s]

train (4/20) loss: 0.03854317022562028 | loss/mean: 0.03854317022562028 | loss/std: 0.003934854902322897 | lr: 0.02 | momentum: 0.9


4/20 * Epoch (valid):   0%|          | 0/313 [00:00<?, ?it/s]

valid (4/20) loss: 0.037654136556386955 | loss/mean: 0.037654136556386955 | loss/std: 0.004379289860023009 | lr: 0.02 | momentum: 0.9
* Epoch (4/20) 


5/20 * Epoch (train):   0%|          | 0/1875 [00:00<?, ?it/s]

In [ ]:
input_image = valid_fashion_mnist_data[0][0].numpy() 
predicted_image = cnn_autoencoder( 
    torch.unsqueeze(valid_fashion_mnist_data[0][0], 0) 
) 
predicted_image = predicted_image.detach().numpy( ).squeeze(0).squeeze(0) 
f, axarr = plt.subplots(2,1,  figsize=(5, 5)) 
axarr[0].imshow(predicted_image, cmap='gray') 
axarr[1].imshow(input_image.squeeze(0), cmap='gray') 
 